In [ ]:
#This notebook will make a Word2Vec embedding using CBoW method
#Training corpus = 4 sentences
#For each sentence form input, output with window of 1
#Logic Read the Training corpus (having grand total of 4 sentences)
#Form a wordToIndex map : {word:  index}, and IndexToWord map {index:word}
#Use the WordToIndex map to form a OHE
# #of Dimensions = 6
#For each sentence do:
    # extract the context and target pair (also same as input, output)
    # Convert them to OHE
    # Add them to x_train and y_train
# Feed the x_train and y_train to the Keras NN for an epoch of 100
# At the the end extract the weights of hidden layer. That will act as word embedding    

In [122]:
f = open("w2v.txt")
text =f.read()
text = text.lower()
training_corpus = text.split(".")
print (len(training_corpus))
for i in range(0, len(training_corpus), 10):
    print (training_corpus[i])

27
in linguistics word embeddings were discussed in the research area of distributional semantics
 the area developed gradually and really took off after 2010, partly because important advances had been made since then on the quality of vectors and the training speed of the model
 the results presented by asgari and mofrad suggest that biovectors can characterize biological sequences in terms of biochemical and biophysical interpretations of the underlying patterns


In [1]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential
from keras.preprocessing import sequence

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [83]:
training_corpus = [
    "Today is a great day filled with blue sky EOS",
    "I am here to make my day EOS",
    "I need to fill up the gas for the long drive EOS",
    "Love driving in clear skies EOS"
]

In [123]:
def FormUniqueIndexAndVectorizeInput(tc):
    WordToInd = {}
    IndToWord = {}
    vect_tc = []    
    ind = 0
    for sent in tc:
        print (sent)
        words = sent.split()
        ind_tc= []
        for s in words:
            if s not in WordToInd:
                WordToInd[s] = ind
                IndToWord[ind] = s
                ind += 1
            ind_tc.append(WordToInd[s])
        vect_tc.append(ind_tc)
    return WordToInd, IndToWord, vect_tc

WordToInd, IndToWord, vect_tc = FormUniqueIndexAndVectorizeInput(training_corpus)
print (len(WordToInd))

in linguistics word embeddings were discussed in the research area of distributional semantics
 it aims to quantify and categorize semantic similarities between linguistic items based on their distributional properties in large samples of language data
 the underlying idea that "a word is characterized by the company it keeps" was popularized by firth


the technique of representing words as vectors has roots in the 1960s with the development of the vector space model for information retrieval
 reducing the number of dimensions using singular value decomposition then led to the introduction of latent semantic analysis in the late 1980s
in 2000 bengio et al
 provided in a series of papers the "neural probabilistic language models" to reduce the high dimensionality of words representations in contexts by "learning a distributed representation for words"
 (bengio et al, 2003)
 word embeddings come in two different styles, one in which words are expressed as vectors of co-occurring words, 

In [149]:
import numpy as np
def CBoWInput(vtc, window=2):
    train = []
    for sent in vtc:
        for i,word in enumerate(sent):
            if (i < len(sent) -1):
                j = 0
                while j< window:
                    train.append ((sent[i-j], sent[i+1]))
                    j += 1
                    if(i - j < 0):
                        break
        #print ("for sentence=", sent, "training set=",train)
    x_train = np.ndarray((len(train), 1))
    y_train = np.ndarray((len(train), 1))
    for i,t in enumerate(train):
        x_train[i] = t[0]
        y_train[i] = t[1]

    return train, x_train, y_train
training_data, x, y = CBoWInput(vect_tc)
#for t in training_data:
 #   print (IndToWord[t[0]], IndToWord[t[1]])

In [150]:
x.shape

(993, 1)

In [126]:
#create Model
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import Sequential

modelNN=Sequential()

modelNN.add(Dense(6, input_shape=(len(WordToInd),), activation = "relu"))
modelNN.add(Dense(len(WordToInd), activation='softmax'))
modelNN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelNN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 6)                 1746      
_________________________________________________________________
dense_30 (Dense)             (None, 290)               2030      
Total params: 3,776
Trainable params: 3,776
Non-trainable params: 0
_________________________________________________________________
None


In [151]:
from keras.utils import to_categorical
batch_size=28
num_epochs = 500
x_enc = to_categorical(x, num_classes=len(WordToInd))
y_enc = to_categorical(y, num_classes=len(WordToInd))
#print (x_enc.shape, y_enc.shape)
#print (x_enc[0], x_enc[1])


In [131]:
modelNN.fit(x_enc, y_enc,  batch_size=batch_size, epochs=num_epochs)

Epoch 1/500
993/993 [==============================] - 0s 146us/step - loss: 2.7800 - acc: 0.2800
Epoch 2/500
993/993 [==============================] - 0s 137us/step - loss: 2.7780 - acc: 0.2769
Epoch 3/500
993/993 [==============================] - 0s 143us/step - loss: 2.7745 - acc: 0.2820
Epoch 4/500
993/993 [==============================] - 0s 150us/step - loss: 2.7715 - acc: 0.2840
Epoch 5/500
993/993 [==============================] - 0s 147us/step - loss: 2.7686 - acc: 0.2830
Epoch 6/500
993/993 [==============================] - 0s 157us/step - loss: 2.7662 - acc: 0.2810
Epoch 7/500
993/993 [==============================] - 0s 150us/step - loss: 2.7628 - acc: 0.2820
Epoch 8/500
993/993 [==============================] - 0s 148us/step - loss: 2.7601 - acc: 0.2830
Epoch 9/500
993/993 [==============================] - 0s 150us/step - loss: 2.7572 - acc: 0.2870
Epoch 10/500
993/993 [==============================] - 0s 150us/step - loss: 2.7551 - acc: 0.2820
Epoch 11/500
993/99

993/993 [==============================] - 0s 124us/step - loss: 2.5956 - acc: 0.2860
Epoch 84/500
993/993 [==============================] - 0s 135us/step - loss: 2.5946 - acc: 0.2880
Epoch 85/500
993/993 [==============================] - 0s 135us/step - loss: 2.5925 - acc: 0.2900
Epoch 86/500
993/993 [==============================] - 0s 126us/step - loss: 2.5901 - acc: 0.2880
Epoch 87/500
993/993 [==============================] - 0s 124us/step - loss: 2.5887 - acc: 0.2860
Epoch 88/500
993/993 [==============================] - 0s 121us/step - loss: 2.5874 - acc: 0.2860
Epoch 89/500
993/993 [==============================] - 0s 122us/step - loss: 2.5852 - acc: 0.2870
Epoch 90/500
993/993 [==============================] - 0s 125us/step - loss: 2.5838 - acc: 0.2870
Epoch 91/500
993/993 [==============================] - 0s 121us/step - loss: 2.5820 - acc: 0.2931
Epoch 92/500
993/993 [==============================] - 0s 135us/step - loss: 2.5802 - acc: 0.2880
Epoch 93/500
993/993 [=

993/993 [==============================] - 0s 131us/step - loss: 2.4777 - acc: 0.2860
Epoch 165/500
993/993 [==============================] - 0s 105us/step - loss: 2.4765 - acc: 0.2971
Epoch 166/500
993/993 [==============================] - 0s 105us/step - loss: 2.4757 - acc: 0.2880
Epoch 167/500
993/993 [==============================] - 0s 104us/step - loss: 2.4736 - acc: 0.2941
Epoch 168/500
993/993 [==============================] - 0s 115us/step - loss: 2.4727 - acc: 0.2890
Epoch 169/500
993/993 [==============================] - 0s 99us/step - loss: 2.4714 - acc: 0.2910
Epoch 170/500
993/993 [==============================] - 0s 100us/step - loss: 2.4709 - acc: 0.2910
Epoch 171/500
993/993 [==============================] - 0s 101us/step - loss: 2.4693 - acc: 0.2890
Epoch 172/500
993/993 [==============================] - 0s 100us/step - loss: 2.4675 - acc: 0.2870
Epoch 173/500
993/993 [==============================] - 0s 101us/step - loss: 2.4669 - acc: 0.2920
Epoch 174/500
9

993/993 [==============================] - 0s 234us/step - loss: 2.3925 - acc: 0.2981
Epoch 247/500
993/993 [==============================] - 0s 208us/step - loss: 2.3911 - acc: 0.3011
Epoch 248/500
993/993 [==============================] - ETA: 0s - loss: 2.3530 - acc: 0.311 - 0s 251us/step - loss: 2.3902 - acc: 0.3001
Epoch 249/500
993/993 [==============================] - 0s 228us/step - loss: 2.3887 - acc: 0.2951
Epoch 250/500
993/993 [==============================] - 0s 176us/step - loss: 2.3888 - acc: 0.2961
Epoch 251/500
993/993 [==============================] - 0s 190us/step - loss: 2.3878 - acc: 0.3021
Epoch 252/500
993/993 [==============================] - 0s 238us/step - loss: 2.3872 - acc: 0.2961
Epoch 253/500
993/993 [==============================] - 0s 268us/step - loss: 2.3858 - acc: 0.3082
Epoch 254/500
993/993 [==============================] - 0s 207us/step - loss: 2.3846 - acc: 0.3001
Epoch 255/500
993/993 [==============================] - 0s 179us/step - los

993/993 [==============================] - 0s 171us/step - loss: 2.3279 - acc: 0.3021
Epoch 328/500
993/993 [==============================] - 0s 125us/step - loss: 2.3273 - acc: 0.3041
Epoch 329/500
993/993 [==============================] - 0s 136us/step - loss: 2.3267 - acc: 0.3051
Epoch 330/500
993/993 [==============================] - 0s 132us/step - loss: 2.3259 - acc: 0.2971
Epoch 331/500
993/993 [==============================] - 0s 123us/step - loss: 2.3255 - acc: 0.2981
Epoch 332/500
993/993 [==============================] - 0s 129us/step - loss: 2.3241 - acc: 0.3041
Epoch 333/500
993/993 [==============================] - 0s 126us/step - loss: 2.3237 - acc: 0.2961
Epoch 334/500
993/993 [==============================] - 0s 170us/step - loss: 2.3230 - acc: 0.2981
Epoch 335/500
993/993 [==============================] - 0s 150us/step - loss: 2.3221 - acc: 0.2961
Epoch 336/500
993/993 [==============================] - 0s 133us/step - loss: 2.3212 - acc: 0.3021
Epoch 337/500


993/993 [==============================] - 0s 147us/step - loss: 2.2766 - acc: 0.2880
Epoch 409/500
993/993 [==============================] - 0s 132us/step - loss: 2.2763 - acc: 0.3001
Epoch 410/500
993/993 [==============================] - 0s 119us/step - loss: 2.2752 - acc: 0.3011
Epoch 411/500
993/993 [==============================] - 0s 132us/step - loss: 2.2749 - acc: 0.2951
Epoch 412/500
993/993 [==============================] - 0s 122us/step - loss: 2.2747 - acc: 0.3001
Epoch 413/500
993/993 [==============================] - 0s 113us/step - loss: 2.2736 - acc: 0.3001
Epoch 414/500
993/993 [==============================] - 0s 128us/step - loss: 2.2732 - acc: 0.3011
Epoch 415/500
993/993 [==============================] - 0s 122us/step - loss: 2.2727 - acc: 0.2941
Epoch 416/500
993/993 [==============================] - 0s 138us/step - loss: 2.2724 - acc: 0.2931
Epoch 417/500
993/993 [==============================] - 0s 122us/step - loss: 2.2719 - acc: 0.2890
Epoch 418/500


993/993 [==============================] - 0s 146us/step - loss: 2.2340 - acc: 0.3001
Epoch 490/500
993/993 [==============================] - 0s 135us/step - loss: 2.2346 - acc: 0.2910
Epoch 491/500
993/993 [==============================] - 0s 133us/step - loss: 2.2353 - acc: 0.2931
Epoch 492/500
993/993 [==============================] - 0s 128us/step - loss: 2.2332 - acc: 0.2941
Epoch 493/500
993/993 [==============================] - 0s 130us/step - loss: 2.2330 - acc: 0.2981
Epoch 494/500
993/993 [==============================] - 0s 137us/step - loss: 2.2319 - acc: 0.2981
Epoch 495/500
993/993 [==============================] - 0s 148us/step - loss: 2.2327 - acc: 0.2890
Epoch 496/500
993/993 [==============================] - 0s 128us/step - loss: 2.2314 - acc: 0.3001
Epoch 497/500
993/993 [==============================] - 0s 137us/step - loss: 2.2303 - acc: 0.3001
Epoch 498/500
993/993 [==============================] - 0s 140us/step - loss: 2.2303 - acc: 0.2991
Epoch 499/500


In [132]:
w2vLayer = modelNN.layers[0]
print (w2vLayer.name)
wts = w2vLayer.get_weights()
#wts[0]== Weight, wts[1]= bias
print (len(wts), len(wts[0]))
for i,w in enumerate(wts[0]):
    print (i, w)

dense_29
2 290
0 [-0.88929856 -0.62975734  2.322333    2.2121503  -0.24275294 -1.2494944 ]
1 [ 0.75201565  0.04096923  3.6166577  -0.862813    3.2255642   3.0097854 ]
2 [-0.29101992  2.0308578   0.93633074 -1.6293656  -0.6627589   2.6122117 ]
3 [-0.14685006  3.8080506   0.58657104 -0.58574474 -1.1792847   1.1342607 ]
4 [ 3.6231878   4.9606915   0.28368974  0.59521115 -1.088758    0.2781875 ]
5 [1.0971725  4.8936467  0.86028063 3.7351582  1.6322545  0.2734988 ]
6 [-0.21463598 -1.0257059  -1.0212944   2.3307908   1.0270495  -1.5242561 ]
7 [ 3.0520196  -1.6872501  -1.4783219   3.2657852   1.6561127  -0.34780368]
8 [ 2.8322997 -0.3983371 -1.2302314  0.2859466  5.3976007 -0.7820187]
9 [-0.83763653 -0.07642663 -0.31995726 -0.39942712  2.1646383   0.21640038]
10 [ 1.8320295  -0.38855258 -0.01479468  0.05653698  0.01409547  0.23713581]
11 [ 0.10100773 -0.05692254  0.13033839 -0.03017608  0.10222936 -0.12685558]
12 [-1.786692  -1.4301419  4.3195915 -1.4321048  1.6624048 -1.0742301]
13 [-1.71675

In [133]:
for k,v in WordToInd.items():
    print (k,v, wts[0][v])

in 0 [-0.88929856 -0.62975734  2.322333    2.2121503  -0.24275294 -1.2494944 ]
linguistics 1 [ 0.75201565  0.04096923  3.6166577  -0.862813    3.2255642   3.0097854 ]
word 2 [-0.29101992  2.0308578   0.93633074 -1.6293656  -0.6627589   2.6122117 ]
embeddings 3 [-0.14685006  3.8080506   0.58657104 -0.58574474 -1.1792847   1.1342607 ]
were 4 [ 3.6231878   4.9606915   0.28368974  0.59521115 -1.088758    0.2781875 ]
discussed 5 [1.0971725  4.8936467  0.86028063 3.7351582  1.6322545  0.2734988 ]
the 6 [-0.21463598 -1.0257059  -1.0212944   2.3307908   1.0270495  -1.5242561 ]
research 7 [ 3.0520196  -1.6872501  -1.4783219   3.2657852   1.6561127  -0.34780368]
area 8 [ 2.8322997 -0.3983371 -1.2302314  0.2859466  5.3976007 -0.7820187]
of 9 [-0.83763653 -0.07642663 -0.31995726 -0.39942712  2.1646383   0.21640038]
distributional 10 [ 1.8320295  -0.38855258 -0.01479468  0.05653698  0.01409547  0.23713581]
semantics 11 [ 0.10100773 -0.05692254  0.13033839 -0.03017608  0.10222936 -0.12685558]
it 12 

In [147]:
print ("expressed", wts[0][WordToInd["expressed"]])
print ( "proposed", wts[0][WordToInd["proposed"]])
print ( "semantics", wts[0][WordToInd["semantics"]])
print ( "semantic", wts[0][WordToInd["semantic"]])

expressed [ 2.0147498   3.1835005  -1.577108   -0.58519655  6.1259494   1.805676  ]
proposed [-0.63978356  2.8025792   4.313328    2.8522978  -1.8218298  -1.0740772 ]
semantics [ 0.10100773 -0.05692254  0.13033839 -0.03017608  0.10222936 -0.12685558]
semantic [ 0.9327205  -1.8306663  -1.0781221  -0.41344702  1.7052569  -0.37389767]
